In [25]:
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Fraud Detection Hackathon with Privacy Village #DPD25FEST")

tls_cert_name: ip-100-100-15-185.eu-west-1.compute.internal_11589e76-6316-4116-a289-97064b78daf5
cert_thumbprint: d90875bf23925e6fcad143593f877711241236f409afc7d935fb60ce8afd65154f67bca43e0d829fb70948dfdb74a4aee265362e9f5a436ef478c6ae8fc29348
server_hostname: ip-100-100-15-185.eu-west-1.compute.internal
local_host_port: 11589e76-6316-4116-a289-97064b78daf5


IOStream.flush timed out


Dataset "Transaction Fraud Hackathon Dataset" loaded to the kernel as transaction_fraud_hackathon_dataset
Key Name                       Value Type     
---------------------------------------------
test_x                         DataFrame      
train_x                        PrivateDataFrame
train_y                        PrivateDataFrame

Connected to Antigranular server session id: 5e17ee57-103c-41e8-98e9-7d1cc498e9de, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [26]:
%%ag
x_train = transaction_fraud_hackathon_dataset["train_x"]
y_train = transaction_fraud_hackathon_dataset["train_y"]
x_test = transaction_fraud_hackathon_dataset["test_x"]

In [27]:
%%ag
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten
from tensorflow.keras.layers import Dense, Dropout
from op_tensorflow import PrivateKerasModel, PrivateDataLoader
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from op_pandas import standard_scaler, PrivateDataFrame, train_test_split

# Normal keras model
#seqM = Sequential([
#    Dense(64, activation='relu', input_shape=(9,)), # 9 = Lenght of fraud attrs
#    Dropout(0.3),
#    Dense(32, activation='relu'),
#    Dropout(0.3),
#    Dense(1, activation='sigmoid')  # 1 = Binary classification
#])

model = Sequential([
    # Giriş katmanı: 9 özellik
    Dense(512, activation='tanh', input_shape=(9,)),
    BatchNormalization(),
    Dropout(0.1),
    Dense(256, activation='tanh'),
    BatchNormalization(),
    Dropout(0.1),
    Dense(128, activation='tanh'),
    Dropout(0.1),
    BatchNormalization(),
    Dense(128, activation='tanh'),
    BatchNormalization(),
    Dense(128, activation='tanh'),
    BatchNormalization(),
    Dense(128, activation='tanh'),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
])

# Create DP keras model
dp_model = PrivateKerasModel(model=model, l2_norm_clip=1, noise_multiplier=1)

# Use a standard (non-DP) optimizer directly from keras
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0003) #0.0003

# PrivateKerasModel uses similar API as standard Keras, we compile
dp_model.compile(
	optimizer = optimizer,
	loss = 'binary_crossentropy',
	metrics = ["accuracy"]
)

/usr/local/lib/python3.10/site-packages/tensorflow/__init__.py:30: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils as _distutils
2025-02-27 12:41:12.972773: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.

2025-02-27 12:41:12.975863: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-27 12:41:13.161973: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-27 12:41:13.162014: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unabl

In [28]:
%%ag
data_loader = PrivateDataLoader(feature_df=x_train , label_df=y_train, batch_size=512)#128 256

In [29]:
%%ag
dp_model.fit(x=data_loader, epochs=20, target_delta=1)

Epoch 1/20

69/69 - 24s - loss: 0.6429 - accuracy: 0.6423 - 24s/epoch - 342ms/step

Epoch 2/20

69/69 - 13s - loss: 0.6133 - accuracy: 0.6631 - 13s/epoch - 194ms/step

Epoch 3/20

69/69 - 13s - loss: 0.6055 - accuracy: 0.6749 - 13s/epoch - 190ms/step

Epoch 4/20

69/69 - 13s - loss: 0.6016 - accuracy: 0.6780 - 13s/epoch - 196ms/step

Epoch 5/20

69/69 - 13s - loss: 0.5981 - accuracy: 0.6805 - 13s/epoch - 194ms/step

Epoch 6/20

69/69 - 13s - loss: 0.5925 - accuracy: 0.6854 - 13s/epoch - 190ms/step

Epoch 7/20

69/69 - 14s - loss: 0.5879 - accuracy: 0.6883 - 14s/epoch - 199ms/step

Epoch 8/20

69/69 - 13s - loss: 0.5886 - accuracy: 0.6852 - 13s/epoch - 194ms/step

Epoch 9/20

69/69 - 13s - loss: 0.5871 - accuracy: 0.6901 - 13s/epoch - 193ms/step

Epoch 10/20

69/69 - 13s - loss: 0.5834 - accuracy: 0.6923 - 13s/epoch - 190ms/step

Epoch 11/20

69/69 - 13s - loss: 0.5821 - accuracy: 0.6943 - 13s/epoch - 193ms/step

Epoch 12/20

69/69 - 13s - loss: 0.5837 - accuracy: 0.6934 - 13s/epoch - 1

In [30]:
%%ag
from op_pandas import standard_scaler, PrivateDataFrame
x_test=PrivateDataFrame(x_test)

In [31]:
%%ag
y_pred = dp_model.predict(x_test, label_columns=["output"])
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))
result = submit_predictions(y_pred)

469/469 [==============================] - 5s 9ms/step

score: {'leaderboard': 0.6869986528842627, 'logs': {'BIN_ACC': 0.6869986528842627, 'LIN_EPS': -0.0}}



aşşa farklı

In [32]:
%%ag
dp_model.fit(x=data_loader, epochs=20, target_delta=1)

Epoch 1/20

69/69 - 13s - loss: 0.5725 - accuracy: 0.7025 - 13s/epoch - 196ms/step

Epoch 2/20

69/69 - 13s - loss: 0.5694 - accuracy: 0.7048 - 13s/epoch - 194ms/step

Epoch 3/20

69/69 - 13s - loss: 0.5665 - accuracy: 0.7052 - 13s/epoch - 194ms/step

Epoch 4/20

69/69 - 13s - loss: 0.5693 - accuracy: 0.7047 - 13s/epoch - 194ms/step

Epoch 5/20

69/69 - 13s - loss: 0.5635 - accuracy: 0.7074 - 13s/epoch - 191ms/step

Epoch 6/20

69/69 - 13s - loss: 0.5644 - accuracy: 0.7074 - 13s/epoch - 193ms/step

Epoch 7/20

69/69 - 13s - loss: 0.5674 - accuracy: 0.7034 - 13s/epoch - 190ms/step

Epoch 8/20

69/69 - 13s - loss: 0.5625 - accuracy: 0.7075 - 13s/epoch - 193ms/step

Epoch 9/20

69/69 - 13s - loss: 0.5637 - accuracy: 0.7076 - 13s/epoch - 194ms/step

Epoch 10/20

69/69 - 13s - loss: 0.5651 - accuracy: 0.7071 - 13s/epoch - 193ms/step

Epoch 11/20

69/69 - 13s - loss: 0.5603 - accuracy: 0.7110 - 13s/epoch - 191ms/step

Epoch 12/20

69/69 - 13s - loss: 0.5560 - accuracy: 0.7132 - 13s/epoch - 1

In [33]:
%%ag
y_pred = dp_model.predict(x_test, label_columns=["output"])
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))
result = submit_predictions(y_pred)

469/469 [==============================] - 4s 9ms/step

score: {'leaderboard': 0.6868932771554894, 'logs': {'BIN_ACC': 0.6868932771554894, 'LIN_EPS': -0.0}}



farklı

In [34]:
%%ag
dp_model.fit(x=data_loader, epochs=5, target_delta=1)

Epoch 1/5

69/69 - 13s - loss: 0.5503 - accuracy: 0.7207 - 13s/epoch - 194ms/step

Epoch 2/5

69/69 - 13s - loss: 0.5451 - accuracy: 0.7213 - 13s/epoch - 191ms/step

Epoch 3/5

69/69 - 13s - loss: 0.5495 - accuracy: 0.7213 - 13s/epoch - 191ms/step

Epoch 4/5

69/69 - 13s - loss: 0.5447 - accuracy: 0.7257 - 13s/epoch - 193ms/step

Epoch 5/5

69/69 - 13s - loss: 0.5467 - accuracy: 0.7216 - 13s/epoch - 191ms/step

message: Error ID is - fac4f014-3089-4f5e-a93b-990d18306ffc - Error from upstream service: Client error '400 Bad Request' for url 'http://supervisor-prod-private.antigranular.com/privacyRequest'
For more information check: https://httpstatuses.com/400. Response text: {"detail":"Maximum delta or epsilon exceeded. Your current balance left is eps : 10.0 and delta : 0.001"}

message: Error ID is - 75499265-63ca-47a0-af2e-7f18a4f2fc91 - Error from upstream service: Client error '400 Bad Request' for url 'http://supervisor-prod-private.antigranular.com/privacyRequest'
For more inform

In [35]:
%%ag
y_pred = dp_model.predict(x_test, label_columns=["output"])
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))
result = submit_predictions(y_pred)

469/469 [==============================] - 4s 9ms/step

error: Submission limit exceeded. Only 1 submission per 5 minutes and 100 submissions per 24 hours are allowed.



farklı

In [36]:
%%ag

optimizer = tf.keras.optimizers.Adam(learning_rate=0.00003)

# PrivateKerasModel uses similar API as standard Keras, we compile
dp_model.compile(
	optimizer = optimizer,
	loss = 'binary_crossentropy',
	metrics = ["accuracy"]
)

In [37]:
%%ag
dp_model.fit(x=data_loader, epochs=20, target_delta=1)


Epoch 1/20

69/69 - 24s - loss: 0.5417 - accuracy: 0.7250 - 24s/epoch - 351ms/step

Epoch 2/20

69/69 - 13s - loss: 0.5422 - accuracy: 0.7251 - 13s/epoch - 196ms/step

Epoch 3/20

69/69 - 13s - loss: 0.5426 - accuracy: 0.7245 - 13s/epoch - 191ms/step

Epoch 4/20

69/69 - 13s - loss: 0.5382 - accuracy: 0.7267 - 13s/epoch - 194ms/step

Epoch 5/20

69/69 - 13s - loss: 0.5395 - accuracy: 0.7247 - 13s/epoch - 193ms/step

Epoch 6/20

69/69 - 13s - loss: 0.5372 - accuracy: 0.7299 - 13s/epoch - 193ms/step

Epoch 7/20

69/69 - 13s - loss: 0.5393 - accuracy: 0.7284 - 13s/epoch - 193ms/step

Epoch 8/20

69/69 - 13s - loss: 0.5382 - accuracy: 0.7301 - 13s/epoch - 194ms/step

Epoch 9/20

69/69 - 13s - loss: 0.5349 - accuracy: 0.7301 - 13s/epoch - 187ms/step

Epoch 10/20

69/69 - 13s - loss: 0.5364 - accuracy: 0.7283 - 13s/epoch - 191ms/step

Epoch 11/20

69/69 - 13s - loss: 0.5362 - accuracy: 0.7288 - 13s/epoch - 194ms/step

Epoch 12/20

69/69 - 13s - loss: 0.5355 - accuracy: 0.7305 - 13s/epoch - 1

In [38]:
%%ag
y_pred = dp_model.predict(x_test, label_columns=["output"])
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))
result = submit_predictions(y_pred)

469/469 [==============================] - 5s 9ms/step

score: {'leaderboard': 0.6895784046879389, 'logs': {'BIN_ACC': 0.6895784046879389, 'LIN_EPS': -0.0}}



In [39]:
%%ag
dp_model.fit(x=data_loader, epochs=20, target_delta=1)


Epoch 1/20

69/69 - 13s - loss: 0.5342 - accuracy: 0.7312 - 13s/epoch - 193ms/step

Epoch 2/20

69/69 - 14s - loss: 0.5413 - accuracy: 0.7247 - 14s/epoch - 197ms/step

Epoch 3/20

69/69 - 13s - loss: 0.5389 - accuracy: 0.7257 - 13s/epoch - 191ms/step

Epoch 4/20

69/69 - 13s - loss: 0.5389 - accuracy: 0.7264 - 13s/epoch - 193ms/step

Epoch 5/20

69/69 - 13s - loss: 0.5386 - accuracy: 0.7282 - 13s/epoch - 194ms/step

Epoch 6/20

69/69 - 13s - loss: 0.5378 - accuracy: 0.7283 - 13s/epoch - 191ms/step

Epoch 7/20

69/69 - 13s - loss: 0.5348 - accuracy: 0.7292 - 13s/epoch - 194ms/step

Epoch 8/20

69/69 - 13s - loss: 0.5285 - accuracy: 0.7329 - 13s/epoch - 190ms/step

Epoch 9/20

69/69 - 13s - loss: 0.5345 - accuracy: 0.7307 - 13s/epoch - 191ms/step

Epoch 10/20

69/69 - 13s - loss: 0.5354 - accuracy: 0.7308 - 13s/epoch - 194ms/step

Epoch 11/20

69/69 - 13s - loss: 0.5340 - accuracy: 0.7305 - 13s/epoch - 196ms/step

Epoch 12/20

69/69 - 13s - loss: 0.5367 - accuracy: 0.7281 - 13s/epoch - 1

In [40]:
%%ag
y_pred = dp_model.predict(x_test, label_columns=["output"])
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))
result = submit_predictions(y_pred)

469/469 [==============================] - 4s 9ms/step

score: {'leaderboard': 0.6901600900468811, 'logs': {'BIN_ACC': 0.6901600900468811, 'LIN_EPS': -0.0}}



In [41]:
%%ag
dp_model.fit(x=data_loader, epochs=20, target_delta=1)


Epoch 1/20

69/69 - 13s - loss: 0.5311 - accuracy: 0.7324 - 13s/epoch - 194ms/step

Epoch 2/20

69/69 - 13s - loss: 0.5314 - accuracy: 0.7346 - 13s/epoch - 195ms/step

Epoch 3/20

69/69 - 14s - loss: 0.5344 - accuracy: 0.7302 - 14s/epoch - 198ms/step

Epoch 4/20

69/69 - 13s - loss: 0.5307 - accuracy: 0.7329 - 13s/epoch - 193ms/step

Epoch 5/20

69/69 - 13s - loss: 0.5321 - accuracy: 0.7338 - 13s/epoch - 194ms/step

Epoch 6/20

69/69 - 13s - loss: 0.5292 - accuracy: 0.7343 - 13s/epoch - 191ms/step

Epoch 7/20

69/69 - 13s - loss: 0.5285 - accuracy: 0.7373 - 13s/epoch - 190ms/step

Epoch 8/20

69/69 - 13s - loss: 0.5323 - accuracy: 0.7314 - 13s/epoch - 193ms/step

Epoch 9/20

69/69 - 13s - loss: 0.5293 - accuracy: 0.7347 - 13s/epoch - 194ms/step

Epoch 10/20

69/69 - 13s - loss: 0.5328 - accuracy: 0.7309 - 13s/epoch - 188ms/step

Epoch 11/20

69/69 - 13s - loss: 0.5289 - accuracy: 0.7333 - 13s/epoch - 190ms/step

Epoch 12/20

69/69 - 13s - loss: 0.5307 - accuracy: 0.7334 - 13s/epoch - 1

In [42]:
%%ag
y_pred = dp_model.predict(x_test, label_columns=["output"])
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))
result = submit_predictions(y_pred)

469/469 [==============================] - 4s 9ms/step

score: {'leaderboard': 0.6893786616768984, 'logs': {'BIN_ACC': 0.6893786616768984, 'LIN_EPS': -0.0}}



In [43]:
%%ag
dp_model.fit(x=data_loader, epochs=20, target_delta=1)


Epoch 1/20

69/69 - 13s - loss: 0.5334 - accuracy: 0.7325 - 13s/epoch - 194ms/step

Epoch 2/20

69/69 - 13s - loss: 0.5346 - accuracy: 0.7299 - 13s/epoch - 191ms/step

Epoch 3/20

69/69 - 13s - loss: 0.5318 - accuracy: 0.7336 - 13s/epoch - 194ms/step

Epoch 4/20

69/69 - 13s - loss: 0.5310 - accuracy: 0.7373 - 13s/epoch - 190ms/step

Epoch 5/20

69/69 - 13s - loss: 0.5295 - accuracy: 0.7350 - 13s/epoch - 191ms/step

Epoch 6/20

69/69 - 13s - loss: 0.5315 - accuracy: 0.7339 - 13s/epoch - 195ms/step

Epoch 7/20

69/69 - 13s - loss: 0.5296 - accuracy: 0.7344 - 13s/epoch - 191ms/step

Epoch 8/20

69/69 - 13s - loss: 0.5325 - accuracy: 0.7326 - 13s/epoch - 193ms/step

Epoch 9/20

69/69 - 13s - loss: 0.5308 - accuracy: 0.7347 - 13s/epoch - 191ms/step

Epoch 10/20

69/69 - 13s - loss: 0.5263 - accuracy: 0.7354 - 13s/epoch - 194ms/step

Epoch 11/20

69/69 - 13s - loss: 0.5312 - accuracy: 0.7329 - 13s/epoch - 194ms/step

Epoch 12/20

69/69 - 13s - loss: 0.5298 - accuracy: 0.7335 - 13s/epoch - 1

In [44]:
%%ag
y_pred = dp_model.predict(x_test, label_columns=["output"])
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))
result = submit_predictions(y_pred)

469/469 [==============================] - 4s 8ms/step

score: {'leaderboard': 0.6905196825824474, 'logs': {'BIN_ACC': 0.6905196825824474, 'LIN_EPS': -0.0}}



In [45]:
%%ag
dp_model.fit(x=data_loader, epochs=20, target_delta=1)


Epoch 1/20

69/69 - 13s - loss: 0.5288 - accuracy: 0.7372 - 13s/epoch - 193ms/step

Epoch 2/20

69/69 - 13s - loss: 0.5302 - accuracy: 0.7357 - 13s/epoch - 194ms/step

Epoch 3/20

69/69 - 13s - loss: 0.5255 - accuracy: 0.7378 - 13s/epoch - 190ms/step

Epoch 4/20

69/69 - 13s - loss: 0.5261 - accuracy: 0.7370 - 13s/epoch - 193ms/step

Epoch 5/20

69/69 - 13s - loss: 0.5290 - accuracy: 0.7359 - 13s/epoch - 190ms/step

Epoch 6/20

69/69 - 13s - loss: 0.5305 - accuracy: 0.7367 - 13s/epoch - 193ms/step

Epoch 7/20



ConnectionError: Error during code execution on AG Server: Error while sending HTTP request to enclave. Rejection(ServerReply { code: 400, message: [123, 34, 109, 101, 115, 115, 97, 103, 101, 34, 58, 34, 69, 114, 114, 111, 114, 32, 115, 101, 110, 100, 105, 110, 103, 32, 72, 84, 84, 80, 32, 114, 101, 113, 117, 101, 115, 116, 32, 116, 111, 32, 115, 101, 114, 118, 101, 114, 46, 32, 34, 44, 34, 111, 114, 105, 103, 105, 110, 34, 58, 34, 99, 108, 105, 101, 110, 116, 34, 125] })

In [ ]:
%%ag
y_pred = dp_model.predict(x_test, label_columns=["output"])
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))
result = submit_predictions(y_pred)

469/469 [==============================] - 5s 10ms/step

score: {'leaderboard': 0.6867677840904587, 'logs': {'BIN_ACC': 0.6867677840904587, 'LIN_EPS': -0.0}}



In [ ]:
%%ag
dp_model.fit(x=data_loader, epochs=20, target_delta=1)


Epoch 1/20

69/69 - 14s - loss: 0.5849 - accuracy: 0.6890 - 14s/epoch - 207ms/step

Epoch 2/20

69/69 - 14s - loss: 0.5794 - accuracy: 0.6948 - 14s/epoch - 210ms/step

Epoch 3/20

69/69 - 14s - loss: 0.5823 - accuracy: 0.6929 - 14s/epoch - 207ms/step

Epoch 4/20

69/69 - 15s - loss: 0.5778 - accuracy: 0.6986 - 15s/epoch - 211ms/step

Epoch 5/20

69/69 - 14s - loss: 0.5830 - accuracy: 0.6935 - 14s/epoch - 207ms/step

Epoch 6/20

69/69 - 14s - loss: 0.5805 - accuracy: 0.6967 - 14s/epoch - 210ms/step

Epoch 7/20

69/69 - 15s - loss: 0.5835 - accuracy: 0.6925 - 15s/epoch - 214ms/step

Epoch 8/20

69/69 - 15s - loss: 0.5831 - accuracy: 0.6917 - 15s/epoch - 213ms/step

Epoch 9/20

69/69 - 15s - loss: 0.5824 - accuracy: 0.6909 - 15s/epoch - 213ms/step

Epoch 10/20

69/69 - 14s - loss: 0.5839 - accuracy: 0.6966 - 14s/epoch - 207ms/step

Epoch 11/20

69/69 - 15s - loss: 0.5841 - accuracy: 0.6918 - 15s/epoch - 212ms/step

Epoch 12/20

69/69 - 14s - loss: 0.5795 - accuracy: 0.6956 - 14s/epoch - 2

In [ ]:
%%ag
y_pred = dp_model.predict(x_test, label_columns=["output"])
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))
result = submit_predictions(y_pred)

469/469 [==============================] - 4s 9ms/step

score: {'leaderboard': 0.6971035650288575, 'logs': {'BIN_ACC': 0.6971035650288575, 'LIN_EPS': -0.0}}



In [ ]:
%%ag
dp_model.fit(x=data_loader, epochs=50, target_delta=1)


Epoch 1/50

69/69 - 14s - loss: 0.5802 - accuracy: 0.6935 - 14s/epoch - 200ms/step

Epoch 2/50

69/69 - 14s - loss: 0.5758 - accuracy: 0.6990 - 14s/epoch - 200ms/step

Epoch 3/50

69/69 - 14s - loss: 0.5782 - accuracy: 0.6946 - 14s/epoch - 200ms/step

Epoch 4/50

69/69 - 14s - loss: 0.5805 - accuracy: 0.6944 - 14s/epoch - 204ms/step

Epoch 5/50

69/69 - 14s - loss: 0.5771 - accuracy: 0.6987 - 14s/epoch - 200ms/step

Epoch 6/50

69/69 - 14s - loss: 0.5795 - accuracy: 0.6925 - 14s/epoch - 203ms/step

Epoch 7/50

69/69 - 14s - loss: 0.5856 - accuracy: 0.6920 - 14s/epoch - 204ms/step

Epoch 8/50

69/69 - 14s - loss: 0.5789 - accuracy: 0.6968 - 14s/epoch - 203ms/step

Epoch 9/50

69/69 - 14s - loss: 0.5819 - accuracy: 0.6927 - 14s/epoch - 201ms/step

Epoch 10/50

69/69 - 14s - loss: 0.5817 - accuracy: 0.6929 - 14s/epoch - 198ms/step

Epoch 11/50

69/69 - 14s - loss: 0.5776 - accuracy: 0.6945 - 14s/epoch - 201ms/step

Epoch 12/50

69/69 - 14s - loss: 0.5777 - accuracy: 0.6971 - 14s/epoch - 2

In [ ]:
%%ag
y_pred = dp_model.predict(x_test, label_columns=["output"])
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))
result = submit_predictions(y_pred)

469/469 [==============================] - 4s 8ms/step

score: {'leaderboard': 0.693573999918356, 'logs': {'BIN_ACC': 0.693573999918356, 'LIN_EPS': -0.0}}



In [ ]:
%%ag
dp_model.fit(x=data_loader, epochs=50, target_delta=1)


Epoch 1/50

69/69 - 14s - loss: 0.5760 - accuracy: 0.6948 - 14s/epoch - 201ms/step

Epoch 2/50

69/69 - 14s - loss: 0.5746 - accuracy: 0.6986 - 14s/epoch - 198ms/step

Epoch 3/50

69/69 - 14s - loss: 0.5721 - accuracy: 0.7015 - 14s/epoch - 201ms/step

Epoch 4/50

69/69 - 14s - loss: 0.5705 - accuracy: 0.7021 - 14s/epoch - 201ms/step

Epoch 5/50

69/69 - 14s - loss: 0.5764 - accuracy: 0.6978 - 14s/epoch - 200ms/step

Epoch 6/50

69/69 - 14s - loss: 0.5748 - accuracy: 0.7000 - 14s/epoch - 201ms/step

Epoch 7/50

69/69 - 14s - loss: 0.5731 - accuracy: 0.6999 - 14s/epoch - 203ms/step

Epoch 8/50

69/69 - 14s - loss: 0.5716 - accuracy: 0.7016 - 14s/epoch - 200ms/step

Epoch 9/50

69/69 - 14s - loss: 0.5766 - accuracy: 0.6964 - 14s/epoch - 203ms/step

Epoch 10/50

69/69 - 14s - loss: 0.5741 - accuracy: 0.6979 - 14s/epoch - 204ms/step

Epoch 11/50

69/69 - 14s - loss: 0.5748 - accuracy: 0.6995 - 14s/epoch - 207ms/step

Epoch 12/50

69/69 - 14s - loss: 0.5725 - accuracy: 0.6995 - 14s/epoch - 2

In [ ]:
%%ag
y_pred = dp_model.predict(x_test, label_columns=["output"])
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))
result = submit_predictions(y_pred)

469/469 [==============================] - 4s 9ms/step

score: {'leaderboard': 0.6991265425067084, 'logs': {'BIN_ACC': 0.6991265425067084, 'LIN_EPS': -0.0}}



In [ ]:
%%ag
dp_model.fit(x=data_loader, epochs=50, target_delta=1)


Epoch 1/50

69/69 - 14s - loss: 0.5678 - accuracy: 0.7067 - 14s/epoch - 203ms/step

Epoch 2/50

69/69 - 14s - loss: 0.5669 - accuracy: 0.7034 - 14s/epoch - 200ms/step

Epoch 3/50

69/69 - 14s - loss: 0.5696 - accuracy: 0.7051 - 14s/epoch - 201ms/step

Epoch 4/50

69/69 - 14s - loss: 0.5685 - accuracy: 0.7040 - 14s/epoch - 201ms/step

Epoch 5/50

69/69 - 14s - loss: 0.5732 - accuracy: 0.7005 - 14s/epoch - 200ms/step

Epoch 6/50

69/69 - 14s - loss: 0.5706 - accuracy: 0.7009 - 14s/epoch - 201ms/step

Epoch 7/50

69/69 - 14s - loss: 0.5696 - accuracy: 0.7016 - 14s/epoch - 201ms/step

Epoch 8/50

69/69 - 14s - loss: 0.5663 - accuracy: 0.7070 - 14s/epoch - 203ms/step

Epoch 9/50

69/69 - 14s - loss: 0.5665 - accuracy: 0.7058 - 14s/epoch - 201ms/step

Epoch 10/50

69/69 - 14s - loss: 0.5697 - accuracy: 0.7041 - 14s/epoch - 201ms/step

Epoch 11/50

69/69 - 14s - loss: 0.5649 - accuracy: 0.7082 - 14s/epoch - 200ms/step

Epoch 12/50

69/69 - 14s - loss: 0.5655 - accuracy: 0.7077 - 14s/epoch - 1

In [ ]:
%%ag
y_pred = dp_model.predict(x_test, label_columns=["output"])
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))
result = submit_predictions(y_pred)

469/469 [==============================] - 4s 8ms/step

score: {'leaderboard': 0.6990539336199451, 'logs': {'BIN_ACC': 0.6990539336199451, 'LIN_EPS': -0.0}}



In [ ]:
%%ag
dp_model.fit(x=data_loader, epochs=20, target_delta=1)

Epoch 1/20

69/69 - 14s - loss: 0.5652 - accuracy: 0.7060 - 14s/epoch - 203ms/step

Epoch 2/20

69/69 - 14s - loss: 0.5626 - accuracy: 0.7081 - 14s/epoch - 206ms/step

Epoch 3/20

69/69 - 14s - loss: 0.5664 - accuracy: 0.7077 - 14s/epoch - 204ms/step

Epoch 4/20

69/69 - 140s - loss: 0.5663 - accuracy: 0.7045 - 140s/epoch - 2s/step

Epoch 5/20

69/69 - 14s - loss: 0.5681 - accuracy: 0.7013 - 14s/epoch - 203ms/step

Epoch 6/20

69/69 - 14s - loss: 0.5646 - accuracy: 0.7065 - 14s/epoch - 203ms/step

Epoch 7/20

69/69 - 14s - loss: 0.5637 - accuracy: 0.7063 - 14s/epoch - 199ms/step

Epoch 8/20

69/69 - 14s - loss: 0.5657 - accuracy: 0.7033 - 14s/epoch - 203ms/step

Epoch 9/20

69/69 - 14s - loss: 0.5693 - accuracy: 0.7044 - 14s/epoch - 203ms/step

Epoch 10/20

69/69 - 14s - loss: 0.5623 - accuracy: 0.7068 - 14s/epoch - 201ms/step

Epoch 11/20

69/69 - 14s - loss: 0.5698 - accuracy: 0.7012 - 14s/epoch - 203ms/step

Epoch 12/20

69/69 - 14s - loss: 0.5665 - accuracy: 0.7053 - 14s/epoch - 20

In [ ]:
%%ag
y_pred = dp_model.predict(x_test, label_columns=["output"])
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))
result = submit_predictions(y_pred)

469/469 [==============================] - 4s 9ms/step

score: {'leaderboard': 0.690737139423609, 'logs': {'BIN_ACC': 0.690737139423609, 'LIN_EPS': -0.0}}



In [ ]:
%%ag
dp_model.fit(x=data_loader, epochs=5, target_delta=1)

Epoch 1/5

69/69 - 14s - loss: 0.5646 - accuracy: 0.7053 - 14s/epoch - 203ms/step

Epoch 2/5

69/69 - 14s - loss: 0.5663 - accuracy: 0.7046 - 14s/epoch - 203ms/step

Epoch 3/5

69/69 - 14s - loss: 0.5638 - accuracy: 0.7036 - 14s/epoch - 201ms/step

Epoch 4/5

69/69 - 14s - loss: 0.5693 - accuracy: 0.7020 - 14s/epoch - 204ms/step

Epoch 5/5

69/69 - 14s - loss: 0.5652 - accuracy: 0.7059 - 14s/epoch - 204ms/step

message: Error ID is - 0929de0d-5c6d-48ce-a132-faad6d3bb9ab - Error from upstream service: Client error '400 Bad Request' for url 'http://supervisor-prod-private.antigranular.com/privacyRequest'
For more information check: https://httpstatuses.com/400. Response text: {"detail":"Maximum delta or epsilon exceeded. Your current balance left is eps : 10.0 and delta : 0.001"}

message: Error ID is - 02cf1327-89f8-4efa-a304-066843674b19 - Error from upstream service: Client error '400 Bad Request' for url 'http://supervisor-prod-private.antigranular.com/privacyRequest'
For more inform

In [ ]:
%%ag
y_pred = dp_model.predict(x_test, label_columns=["output"])
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))
result = submit_predictions(y_pred)

469/469 [==============================] - 4s 8ms/step

score: {'leaderboard': 0.7041610030909989, 'logs': {'BIN_ACC': 0.7041610030909989, 'LIN_EPS': -0.0}}



In [ ]:
%%ag
dp_model.fit(x=data_loader, epochs=5, target_delta=1)

Epoch 1/5

69/69 - 14s - loss: 0.5644 - accuracy: 0.7072 - 14s/epoch - 203ms/step

Epoch 2/5

69/69 - 14s - loss: 0.5638 - accuracy: 0.7061 - 14s/epoch - 204ms/step

Epoch 3/5

69/69 - 14s - loss: 0.5710 - accuracy: 0.7012 - 14s/epoch - 206ms/step

Epoch 4/5

69/69 - 14s - loss: 0.5667 - accuracy: 0.7045 - 14s/epoch - 203ms/step

Epoch 5/5

69/69 - 14s - loss: 0.5623 - accuracy: 0.7082 - 14s/epoch - 200ms/step

message: Error ID is - d77760ae-21d5-457e-a340-52a865d866af - Error from upstream service: Client error '400 Bad Request' for url 'http://supervisor-prod-private.antigranular.com/privacyRequest'
For more information check: https://httpstatuses.com/400. Response text: {"detail":"Maximum delta or epsilon exceeded. Your current balance left is eps : 10.0 and delta : 0.001"}

message: Error ID is - a7c39561-ae65-4d69-b932-502bd541f1fe - Error from upstream service: Client error '400 Bad Request' for url 'http://supervisor-prod-private.antigranular.com/privacyRequest'
For more inform

In [ ]:
%%ag
y_pred = dp_model.predict(x_test, label_columns=["output"])
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))
result = submit_predictions(y_pred)

469/469 [==============================] - 4s 9ms/step

score: {'leaderboard': 0.7013318671657242, 'logs': {'BIN_ACC': 0.7013318671657242, 'LIN_EPS': -0.0}}



In [ ]:
%%ag
dp_model.fit(x=data_loader, epochs=5, target_delta=1)

Epoch 1/5

69/69 - 15s - loss: 0.5638 - accuracy: 0.7075 - 15s/epoch - 210ms/step

Epoch 2/5

69/69 - 14s - loss: 0.5625 - accuracy: 0.7052 - 14s/epoch - 203ms/step

Epoch 3/5

69/69 - 14s - loss: 0.5636 - accuracy: 0.7063 - 14s/epoch - 207ms/step

Epoch 4/5

69/69 - 14s - loss: 0.5692 - accuracy: 0.7021 - 14s/epoch - 206ms/step

Epoch 5/5

69/69 - 14s - loss: 0.5569 - accuracy: 0.7144 - 14s/epoch - 209ms/step

message: Error ID is - 6c981f74-6d38-4c04-9df3-49a175e7406b - Error from upstream service: Client error '400 Bad Request' for url 'http://supervisor-prod-private.antigranular.com/privacyRequest'
For more information check: https://httpstatuses.com/400. Response text: {"detail":"Maximum delta or epsilon exceeded. Your current balance left is eps : 10.0 and delta : 0.001"}

message: Error ID is - fde7fb3a-f95e-4d07-b7fd-4bb1e23deafb - Error from upstream service: Client error '400 Bad Request' for url 'http://supervisor-prod-private.antigranular.com/privacyRequest'
For more inform

In [ ]:
%%ag
y_pred = dp_model.predict(x_test, label_columns=["output"])
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))
result = submit_predictions(y_pred)

469/469 [==============================] - 5s 10ms/step

score: {'leaderboard': 0.697478534523823, 'logs': {'BIN_ACC': 0.697478534523823, 'LIN_EPS': -0.0}}

